winget install 'Microsoft .NET SDK 7.0'  #Needed. Doesn't work on .Net 8
### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/2/2023 9:00:01 PM
Boot                  Sat 12.2.23 10:36
CpuUsage%             76
RamUsage%             {51}
C:%Free               {55}

